# Motivation
论文原文：[《Deep Interest Evolution Network for Click-Through Rate Prediction》](https://arxiv.org/pdf/1809.03672)

参考学习地址：
* [从DIN到DIEN看阿里CTR算法的进化脉络](https://zhuanlan.zhihu.com/p/78365283)
* [【总结】推荐系统——精排篇【3】DIN/DIEN/BST/DSIN/MIMN/SIM/CAN](https://zhuanlan.zhihu.com/p/433135805)

客户越新的行为越能反映用户当时的兴趣，对于推测之后的欣慰所发挥的作业也越大，而旧的行为发挥作用就弱一点，说明用户的兴趣在不断变化。捕捉到用户兴趣的发展变化，就是DIEN要解决的问题。在DIN中提到用户的兴趣之间相互影响有限（例如，游泳和零食），每个兴趣有其自身的发展趋势，DIEN仍采用了attention机制，捕获与candidate相关商品的兴趣发展路径。

<img style="display: block; margin: 0 auto;" src="../../../assets/images/dien.png" width = "1000" height = "600" alt="DIEN" align=center />
DIEN相对于base模型和DIN，只对用户行为序列User behavior sequence处理做了调整。

## Behavior Layer
将用户浏览过的商品转换为对应embedding，并按浏览时间做排序。

## Interest Extractor Layer
兴趣提取层。论文中提到例如通过引入RNN来挖掘和利用行为序列中的依赖关系，比直接对行为序列做pooling效果好，但商品embedding才是对商品的真实表达，也是对用户兴趣的直接反应，RNN的隐层输出向量未必能真正表达用户兴趣（基于目标损失函数$L_{target}$只能有效预测用户的最终兴趣，而无法有监督的学习历史兴趣状态$h_t(t<T>)$）因此兴趣提取层引入了一个有监督学习，强行将RNN输出隐层与商品embedding发生交互。

图中左侧Auxiliary Loss所示，兴趣提取层采用一个基本GRU单元：
1) 按时间步排列输入商品embedding向量，假设第$t$个时间步输入$e(t)$，GRU输出隐单元$h(t)$，另下一个时间步的输入向量$e(t+1)$作为正样本，随机采样负样本$e(t+1)'$（浏览但未点击），满足$e(t+1)'\neq e(t)$；
2) $h(t)$与正负样本向量分别做内积，定义辅助Loss为：
$$
\begin{align}
L_{aux} = -\frac{1}{N}(\sum_{i=1}^{N}\sum_tlog\sigma(h_t^i,e_b^i[t+1])+log(1-\sigma(h_t^i,\hat{e_b^i}[t+1])))
\end{align}
$$
3) 定义CTR任务的loss函数为CTR的loss和辅助loss相加，作为整个网络的loss进行优化。$\alpha$作为超参数，平衡两个loss权重。$L=L_{target}+\alpha * L_{aux}$

采用辅助loss的方式，当GRU处理较长序列的时候有助于降低反向传播的难度，对于商品embedding的学习也有助益。

## Interest Evolving Layer
兴趣发展层，捕获与candidate相关的兴趣发展模式。理论上在兴趣抽取层的GRU已经可以做出下一个想去状态向量的预测，但由于用户兴趣多样性，可能发生兴趣漂移。例如：用户在一段时间对各种书籍感兴趣，而在另一段时间对各种衣服感兴趣。虽然兴趣可能相互影响，但是每种兴趣都有自己不断演化的过程，比如书籍和衣服的演化路径几乎是独立的，而我们主要关注和目标广告相关的演化路径。目标广告是计算机相关书籍，那么书籍相关的演化路径显然比购买衣服相关的演化路径更重要。

这里的GRU不同于兴趣提取层中的GRU，是将candidate的embedding向量与兴趣提取层中GRU输出的隐向量交互，生成attention分数。
$$
\begin{align}
a_t = \frac{exp(h_tWe_a)}{\sum_{j=1}^Texp(h_jWe_a)}
\end{align}
$$
其中，$h_t$是隐向量，$e_a$是candidate的表示向量，$W$是需要训练的参数矩阵。与DIN不同，这里的attention分数采用softmax做了归一化。

AUGRU（GRU with Attention Update gate）将attention分数与update gate相乘，替换原始的update gate。原始的GRU公式为：
$$
\begin{align}
u_t &= \sigma(W^ui_t+U^uh_{t-1}+b^u), \\
r_t &= \sigma(W^ri_t+U^rh_{t-1}+b^r), \\
\tilde{h_t} &= \tanh(W^hi_t+r_t\circ U^hh_{t-1}+b^h), \\
h_t &= (1-u_t)\circ h_{t-1} + u_t\circ \tilde{h_t}, \\
\end{align}
$$
其中，
* $\sigma$是sigmoid激活函数
* $\circ$是对应元素相乘（element-wise product，哈达玛积）
* $W^u,W^r,W^h\in \R^{n_H\times n_I}$，$U^u,U^r,U^h\in \R^{n_H\times n_H}$是需要学习的矩阵，$n^H$是隐藏层维度，$n^I$是输入Embedding的维度
* i_t=e_b[t]是GRU输入，也是想为序列中的第$t$个Embedding向量
* h_t是GRU网络中第$t$个隐向量状态

AUGRU更新GRU中update gate向量，如果$a_t$比较大，那么$\tilde{u_t}'$（GRU中原始的update gate向量）相对也比较大，更多的保留$\tilde{h_t}'$，而消弱上一步的隐层输出$h_{t-1}'$
$$
\begin{align}
\tilde{u_t}' &= a_t * u_t', \\
h_t' &= (1-\tilde{u_t}')\circ h_{t-1}' + \tilde{u_t}'\circ \tilde{h_t}', \\
\end{align}
$$

论文中除了AUGRU外，还介绍了AIGRU（GRU with attention input）和AGRU（Attention based GRU），但实验效果一般：
* AIGRU直接对兴趣抽取层输出的隐向量$h_t$使用注意力机制，也即作用于input：$i_t'=h_t * a_t$
* AGRU直接使用注意力得分来替代GRU更新门：$h_t'=(1-a_t)*h_{t-1}'+a_t*\tilde{h_t}'$